how to aspect oriented?

TODO: depends on transpiling kernels


- affect - run after the code
- prefect - run before the code
- inflect - make available part of the affection within the code
- reflect - describe the current affection
- infect - replace part of code with result of another
- effect - return a list of affections the occured or will occur if reference is provided

In [ ]:
var path = require('path');
var corePath = path.resolve(path.join(__dirname, 'index.js'));

process.stdin.resume(); //so the program will not close instantly

//do something when app is closing
process.on('exit', findSource);

//catches ctrl+c event
process.on('SIGINT', findSource);

// catches "kill pid" (for example: nodemon restart)
process.on('SIGUSR1', findSource);
process.on('SIGUSR2', findSource);

// create an exception then find the source and rerun with aspects attached
process.on('uncaughtException', findSource);
process.on('unhandledRejection', findSource);

function findSource(num, code, err) {
    var m = module;
    var origin;
    while(m != null && !origin) {
        if(m.children.map(m => m.filename).includes(corePath)
           && m.filename !== corePath) {
            origin = m;
        } else {
            m = m.parent;
        }
    }
    console.log(code + ' - ' + origin.filename)
    if(err && err.message && err.message.includes('starting aspects')) {
        process.kill(process.pid, 'SIGUSR1');
    }
}

try {
    throw new Error('starting aspects');
} catch (e) {
    findSource(0, 'START', e);
}


In [ ]:
var importer = require('../Core');
importer.import('aspect oriented');
console.log(process.pid);

In [ ]:
console.log(process.pid);

In [ ]:
process.env.DEBUG